使用 Vanilla LSTM、
========
"無"上一筆需量加入至模型, "不"使用Sliding Window，
========
"無"加入 Self Attention/Attention 架構於LSTM 模型中
========

In [1]:
# Import necessary libraries [B]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt   # plotting
import seaborn as sns   # plotting heatmap
import statsmodels.api as sm  # seasonal trend decomposition
import datetime
from statsmodels.graphics import tsaplots   # autocorrelation
import pymysql
from tensorflow.keras.optimizers import Adam #
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, BatchNormalization, TimeDistributed, Flatten, Bidirectional
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from keras import optimizers
from time import time
from sklearn import preprocessing, metrics
from joblib import dump, load
from datetime import datetime, timedelta
from scipy.interpolate import lagrange
from meteocalc import Temp, dew_point, heat_index, wind_chill, feels_like

%matplotlib inline
# Import necessary libraries [E]

In [2]:
# 參數定義 [B]
today_object = datetime.now()
today = today_object.strftime("%Y-%m-%d-%H-%M-%S")
temperature_zscore_to_path = './../parameter-storage/zscore/temperature-zscore-{}.bin'.format(today)
pres_zscore_to_path = './../parameter-storage/zscore/pres-zscore-{}.bin'.format(today)
wind_speed_zscore_to_path = './../parameter-storage/zscore/wind_speed-zscore-{}.bin'.format(today)
wind_dir_zscore_to_path = './../parameter-storage/zscore/wind_dir-zscore-{}.bin'.format(today)
max_wind_zscore_to_path = './../parameter-storage/zscore/max_wind-zscore-{}.bin'.format(today)
r24_zscore_to_path = './../parameter-storage/zscore/r24-zscore-{}.bin'.format(today)
huvi_zscore_to_path = './../parameter-storage/zscore/huvi-zscore-{}.bin'.format(today)
apparent_temp_zscore_to_path = './../parameter-storage/zscore/apparent_temp-zscore-{}.bin'.format(today)

T_Min_zscore_to_path = './../parameter-storage/zscore/T_Min-zscore-{}.bin'.format(today)
T_Max_zscore_to_path = './../parameter-storage/zscore/T_Max-zscore-{}.bin'.format(today)
day_of_year_zscore_to_path = './../parameter-storage/zscore/day_of_year-zscore-{}.bin'.format(today)
data_week_zscore_to_path = './../parameter-storage/zscore/data_week-zscore-{}.bin'.format(today)
data_hour_zscore_to_path = './../parameter-storage/zscore/data_hour-zscore-{}.bin'.format(today)
data_minute_zscore_to_path = './../parameter-storage/zscore/data_minute-zscore-{}.bin'.format(today)
data_weekday_zscore_to_path = './../parameter-storage/zscore/data_weekday-zscore-{}.bin'.format(today)
minutes_of_the_day_zscore_to_path = './../parameter-storage/zscore/minutes_of_the_day-zscore-{}.bin'.format(today)
demand_zscore_to_path = './../parameter-storage/zscore/demand-zscore-{}.bin'.format(today)
period_type_zscore_to_path = './../parameter-storage/zscore/period_type_zscore-zscore-{}.bin'.format(today)

simple_LSTM_modal_save_path = './../parameter-storage/modal/simple-LSTM-modal-{}.h5'.format(today)
stacked_LSTM_modal_save_path = './../parameter-storage/modal/stacked-LSTM-modal-{}.h5'.format(today)
bidirectional_LSTM_modal_save_path = './../parameter-storage/modal/bidirectional-LSTM-modal-{}.h5'.format(today)
simple_LSTM_kmeans_modal_save_path = './../parameter-storage/modal/simple-LSTM-kmeans-modal-{}.h5'.format(today)
stacked_LSTM_kmeans_modal_save_path = './../parameter-storage/modal/stacked-LSTM-kmeans-modal-{}.h5'.format(today)
bidirectional_LSTM_kmeans_modal_save_path = './../parameter-storage/modal/bidirectional-LSTM-kmeans-modal-{}.h5'.format(today)

zscore_pid_path = './../parameter-storage/pid/zscore-pid.pid'
modal_pid_path = './../parameter-storage/pid/modal-pid.pid'
stacked_modal_pid_path = './../parameter-storage/pid/stacked-modal-pid.pid'
bidirectional_modal_pid_path = './../parameter-storage/pid/bidirectional-modal-pid.pid'
simple_kmeans_modal_pid_path = './../parameter-storage/pid/simple-kmeans-modal-pid.pid'
stacked_kmeans_modal_pid_path = './../parameter-storage/pid/stacked-kmeans-modal-pid.pid'
bidirectional_kmeans_modal_pid_path = './../parameter-storage/pid/bidirectional-kmeans-modal-pid.pid'

kmeans_pid_path = './../parameter-storage/pid/kmeans-pid.pid'

In [3]:
begin_date = '2013-07-01'
end_date = '2021-06-08'
train_begin_date = '2017-06-08'
train_end_date = '2019-06-07'
test_begin_date = '2019-06-08'
test_end_date = '2020-06-07'
predict_begin_date = '2020-06-08'
predict_end_date = '2021-06-07'
# 參數定義 [E]

In [4]:
# 正規化最大最小值 [B]
temperature_range = np.array([0, 50])
pres_range = np.array([0, 1500])
wind_speed_range = np.array([0, 80])
wind_dir_range = np.array([0, 360])
r24_range = np.array([0, 400])
huvi_range = np.array([0, 30])
day_of_year_range = np.array([1, 366])
week_range = np.array([1, 53])
hour_range = np.array([0, 24])
minute_range = np.array([0, 60])
weekday_range = np.array([1, 7])
minutes_of_the_day_range = np.array([0, 86400])
period_type_range = np.array([0, 2])
demand_range = np.array([0, 700])
# 正規化最大最小值 [B]

In [5]:
# 體感溫度計算 [B]
def c_feel_like(temperature, humidity, wind_speed):
    
    temp_c = Temp(temperature, 'c')
    temp_f = temp_c.convert(temp_c, 'c', 'f')
    humidity_percent =  round(humidity * 100)
    apparent_temp = feels_like(temp_f, humidity_percent, wind_speed) # 溫度為華氏(F), 所以體感溫度也是華氏(F)
    return apparent_temp.convert(apparent_temp, 'f', 'c')
# 體感溫度計算 [E]

In [6]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = '　' ,  user = '　' ,  passwd = "　" ,  db = 'ncku_demand' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `id`, `demand_min`, `demand_quarter`, `Total_value`, `Temperature`, 
                    `humd`, `pres`, `w_DSD`, `w_DIR`, `h_FX`, `24R`, `h_UVI`, `T_Min`, `T_Max`, 
                    `day_of_year`, `data_week`, `data_date`, `data_hour`, `data_minute`, `data_weekday`, 
                    `minutes_of_the_day`, `period_type`, `datetime`
                    FROM `demand_with_weather_data` 
                    WHERE `datetime` >= '{}' AND `datetime` < '{}'
                    ORDER BY `datetime` '''.format(begin_date, end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    timestamp = pd.to_datetime(demand_dataframe.datetime, infer_datetime_format=True).values.astype(float)
    demand_dataframe['timestamp'] = timestamp.tolist()
    demand_dataframe = demand_dataframe.set_index('datetime')
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [7]:
# 刪除不需要的欄位 [B]
demand_dataframe = demand_dataframe.drop(columns=['id', 'demand_min', 'Total_value', 'data_date'])
# 刪除不需要的欄位 [E]

In [8]:
demand_dataframe[demand_dataframe.isnull().any(axis=1)]

,demand_quarter,Temperature,humd,pres,w_DSD,w_DIR,h_FX,24R,h_UVI,T_Min,T_Max,day_of_year,data_week,data_hour,data_minute,data_weekday,minutes_of_the_day,period_type,timestamp
datetime,,,,,,,,,,,,,,,,,,,
2017-08-22 02:00:00,120.8,30.0,0.75,993.0,1.1,NaN,3.0,0,0.0,34.0,26.0,234,34,2,0,2,120,1,1.503367e+18
2017-08-22 02:15:00,118.4,30.0,0.75,993.0,1.1,NaN,3.0,0,0.0,34.0,26.0,234,34,2,15,2,135,1,1.503368e+18
2017-08-22 02:30:00,115.2,30.0,0.75,993.0,1.1,NaN,3.0,0,0.0,34.0,26.0,234,34,2,30,2,150,1,1.503369e+18
2017-08-22 02:45:00,115.6,30.0,0.75,993.0,1.1,NaN,3.0,0,0.0,34.0,26.0,234,34,2,45,2,165,1,1.503370e+18
2017-10-11 09:00:00,302.4,33.0,0.60,1008.0,0.7,NaN,4.0,0,5.0,35.0,28.0,284,41,9,0,3,540,2,1.507712e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-01 11:45:00,140.8,28.0,0.55,1011.0,0.9,NaN,5.0,0,4.0,30.0,23.0,306,44,11,45,7,705,2,1.604231e+18
2020-11-04 10:00:00,174.4,27.0,0.48,1013.0,0.9,NaN,4.0,0,5.0,29.0,21.0,309,45,10,0,3,600,2,1.604484e+18
2020-11-04 10:15:00,195.6,27.0,0.48,1013.0,0.9,NaN,4.0,0,5.0,29.0,21.0,309,45,10,15,3,615,2,1.604485e+18


In [9]:
# 補缺失值 [B]
demand_dataframe = demand_dataframe.interpolate(method="linear") 
# 補缺失值 [E]

In [ ]:
# 體感溫度計算 [B]
demand_dataframe['apparent_temp'] = demand_dataframe.apply(lambda demand_dataframe: c_feel_like(demand_dataframe['Temperature'], demand_dataframe['humd'], demand_dataframe['w_DSD']), axis=1)
# 體感溫度計算 [E]

In [11]:
# # 盒箱圖, 看最大最小值等等 [B]
# boxplot = demand_dataframe.boxplot(column=['pres'])
# # 盒箱圖, 看最大最小值等等 [E]

In [12]:
# 清除異常值 [E]
# for x in ['demand_quarter']:
#     q75,q25 = np.percentile(demand_dataframe.loc[:,x],[85,25])
#     intr_qr = q75-q25
 
#     max = q75+(1.5*intr_qr)
#     min = q25-(1.5*intr_qr)
    
#     demand_dataframe.loc[demand_dataframe[x] < min,x] = np.nan
    
#     demand_dataframe.loc[demand_dataframe[x] > max,x] = np.nan
# 清除異常值 [B]

In [ ]:
# 資料正規化 [B]
# 宣告實例 [B]
temperature_zscore = preprocessing.MinMaxScaler()
pres_zscore = preprocessing.MinMaxScaler()
wind_speed_zscore = preprocessing.MinMaxScaler()
wind_dir_zscore = preprocessing.MinMaxScaler()
max_wind_zscore = preprocessing.MinMaxScaler()
r24_zscore = preprocessing.MinMaxScaler()
huvi_zscore = preprocessing.MinMaxScaler()
apparent_temp_zscore = preprocessing.MinMaxScaler()
T_Min_zscore = preprocessing.MinMaxScaler()
T_Max_zscore = preprocessing.MinMaxScaler() 
day_of_year_zscore = preprocessing.MinMaxScaler() 
data_week_zscore = preprocessing.MinMaxScaler() 
data_hour_zscore = preprocessing.MinMaxScaler() 
data_minute_zscore = preprocessing.MinMaxScaler() 
data_weekday_zscore = preprocessing.MinMaxScaler() 
minutes_of_the_day_zscore = preprocessing.MinMaxScaler() 
demand_zscore = preprocessing.MinMaxScaler()
period_type_zscore = preprocessing.MinMaxScaler()
# 宣告實例 [E]
# 資料 reshape [B]
temperature_reshape_data = demand_dataframe['Temperature'].values.reshape(-1, 1)
pres_reshape_data = demand_dataframe['pres'].values.reshape(-1, 1)
wind_speed_reshape_data = demand_dataframe['w_DSD'].values.reshape(-1, 1)
wind_dir_reshape_data = demand_dataframe['w_DIR'].values.reshape(-1, 1)
max_wind_reshape_data = demand_dataframe['h_FX'].values.reshape(-1, 1)
r24_reshape_data = demand_dataframe['24R'].values.reshape(-1, 1)
huvi_reshape_data = demand_dataframe['h_UVI'].values.reshape(-1, 1)
apparent_temp_reshape_data = demand_dataframe['apparent_temp'].values.reshape(-1, 1)
T_Min_reshape_data = demand_dataframe['T_Min'].values.reshape(-1, 1)
T_Max_reshape_data = demand_dataframe['T_Max'].values.reshape(-1, 1)
day_of_year_reshape_data = demand_dataframe['day_of_year'].values.reshape(-1, 1)
data_week_reshape_data = demand_dataframe['data_week'].values.reshape(-1, 1)
data_hour_reshape_data = demand_dataframe['data_hour'].values.reshape(-1, 1)
data_minute_reshape_data = demand_dataframe['data_minute'].values.reshape(-1, 1)
data_weekday_reshape_data = demand_dataframe['data_weekday'].values.reshape(-1, 1)
minutes_of_the_day_reshape_data = demand_dataframe['minutes_of_the_day'].values.reshape(-1, 1)
period_type_zscore_reshape_data = demand_dataframe['period_type'].values.reshape(-1, 1)
demand_reshape_data = demand_dataframe['demand_quarter'].values.reshape(-1, 1)
# 資料 reshape [E]
# 部分計算 [B]
temperature_zscore.partial_fit(temperature_range.reshape(-1, 1))
pres_zscore.partial_fit(pres_range.reshape(-1, 1))
wind_speed_zscore.partial_fit(wind_speed_range.reshape(-1, 1))
wind_dir_zscore.partial_fit(wind_dir_range.reshape(-1, 1))
max_wind_zscore.partial_fit(wind_speed_range.reshape(-1, 1))
r24_zscore.partial_fit(r24_range.reshape(-1, 1))
huvi_zscore.partial_fit(huvi_range.reshape(-1, 1))
apparent_temp_zscore.partial_fit(temperature_range.reshape(-1, 1))
T_Min_zscore.partial_fit(temperature_range.reshape(-1, 1))
T_Max_zscore.partial_fit(temperature_range.reshape(-1, 1))
day_of_year_zscore.partial_fit(day_of_year_range.reshape(-1, 1))
data_week_zscore.partial_fit(week_range.reshape(-1, 1))
data_hour_zscore.partial_fit(hour_range.reshape(-1, 1))
data_minute_zscore.partial_fit(minute_range.reshape(-1, 1))
data_weekday_zscore.partial_fit(weekday_range.reshape(-1, 1))
minutes_of_the_day_zscore.partial_fit(minutes_of_the_day_range.reshape(-1, 1))
period_type_zscore.partial_fit(period_type_range.reshape(-1, 1))
demand_zscore.partial_fit(demand_range.reshape(-1, 1))
# 部分計算 [E]
# 將部分計算得結果先做正規化 [B]
demand_dataframe['Temperature'] = temperature_zscore.fit_transform(temperature_reshape_data)
demand_dataframe['pres'] = pres_zscore.fit_transform(pres_reshape_data)
demand_dataframe['w_DSD'] = wind_speed_zscore.fit_transform(wind_speed_reshape_data)
demand_dataframe['w_DIR'] = wind_dir_zscore.fit_transform(wind_dir_reshape_data)
demand_dataframe['h_FX'] = max_wind_zscore.fit_transform(max_wind_reshape_data)
demand_dataframe['24R'] = r24_zscore.fit_transform(r24_reshape_data)
demand_dataframe['h_UVI'] = huvi_zscore.fit_transform(huvi_reshape_data)
demand_dataframe['apparent_temp'] = apparent_temp_zscore.fit_transform(apparent_temp_reshape_data)
demand_dataframe['T_Min'] = T_Min_zscore.fit_transform(T_Min_reshape_data)
demand_dataframe['T_Max'] = T_Max_zscore.fit_transform(T_Max_reshape_data)
demand_dataframe['day_of_year'] = day_of_year_zscore.fit_transform(day_of_year_reshape_data)
demand_dataframe['data_week'] = data_week_zscore.fit_transform(data_week_reshape_data)
demand_dataframe['data_hour'] = data_hour_zscore.fit_transform(data_hour_reshape_data)
demand_dataframe['data_minute'] = data_minute_zscore.fit_transform(data_minute_reshape_data)
demand_dataframe['data_weekday'] = data_weekday_zscore.fit_transform(data_weekday_reshape_data)
demand_dataframe['minutes_of_the_day'] = minutes_of_the_day_zscore.fit_transform(minutes_of_the_day_reshape_data)
demand_dataframe['period_type'] = period_type_zscore.fit_transform(period_type_zscore_reshape_data)
demand_dataframe['demand_quarter_1'] = demand_zscore.fit_transform(demand_reshape_data)
# 將部分計算得結果先做正規化 [E]
# 資料正規化 [E]

In [14]:
# 儲存正規化模型 [B]
dump(temperature_zscore, temperature_zscore_to_path, compress=True)
dump(pres_zscore, pres_zscore_to_path, compress=True)
dump(wind_speed_zscore, wind_speed_zscore_to_path, compress=True)
dump(wind_dir_zscore,wind_dir_zscore_to_path, compress=True)
dump(max_wind_zscore,max_wind_zscore_to_path, compress=True)
dump(r24_zscore,r24_zscore_to_path, compress=True)
dump(huvi_zscore,huvi_zscore_to_path, compress=True)
dump(apparent_temp_zscore, apparent_temp_zscore_to_path, compress=True)
dump(T_Min_zscore, T_Min_zscore_to_path, compress=True) #儲存參數
dump(T_Max_zscore, T_Max_zscore_to_path, compress=True) #儲存參數
dump(day_of_year_zscore, day_of_year_zscore_to_path, compress=True) #儲存參數
dump(data_week_zscore, data_week_zscore_to_path, compress=True) #儲存參數
dump(data_hour_zscore, data_hour_zscore_to_path, compress=True) #儲存參數
dump(data_minute_zscore, data_minute_zscore_to_path, compress=True) #儲存參數
dump(data_weekday_zscore, data_weekday_zscore_to_path, compress=True) #儲存參數
dump(demand_zscore, demand_zscore_to_path, compress=True) #儲存參數
dump(period_type_zscore, period_type_zscore_to_path, compress=True) #儲存參數
dump(minutes_of_the_day_zscore, minutes_of_the_day_zscore_to_path, compress=True) #儲存參數
# 儲存正規化模型 [E]

['./../parameter-storage/zscore/minutes_of_the_day-zscore-2022-06-02-10-48-41.bin']

In [15]:
# 儲存檔案最後更新檔案日期 [B]
zscore_pid_write = open(zscore_pid_path, 'w', encoding='utf-8')
zscore_pid_write.write(today)
print(today)
# 儲存檔案最後更新檔案日期 [E]

2022-06-02-10-48-41


In [16]:
demand_dataframe

,demand_quarter,Temperature,humd,pres,w_DSD,w_DIR,h_FX,24R,h_UVI,T_Min,...,day_of_year,data_week,data_hour,data_minute,data_weekday,minutes_of_the_day,period_type,timestamp,apparent_temp,demand_quarter_1
datetime,,,,,,,,,,,,,,,,,,,,,
2017-06-08 13:45:00,283.6,0.925926,0.60,0.982439,0.232143,0.583333,0.24,0.0,0.476190,0.92,...,0.432877,0.423077,0.565217,1.000000,0.5,0.578947,1.0,1.496930e+18,0.682901,0.478412
2017-06-08 14:00:00,277.6,0.962963,0.58,0.982439,0.205357,0.694444,0.28,0.0,0.380952,0.92,...,0.432877,0.423077,0.608696,0.000000,0.5,0.589474,1.0,1.496930e+18,0.692003,0.467967
2017-06-08 14:15:00,269.6,0.962963,0.58,0.982439,0.205357,0.694444,0.28,0.0,0.380952,0.92,...,0.432877,0.423077,0.608696,0.333333,0.5,0.600000,1.0,1.496931e+18,0.692003,0.454039
2017-06-08 14:30:00,288.4,0.962963,0.58,0.982439,0.205357,0.694444,0.28,0.0,0.380952,0.92,...,0.432877,0.423077,0.608696,0.666667,0.5,0.610526,1.0,1.496932e+18,0.692003,0.486769
2017-06-08 14:45:00,295.2,0.962963,0.58,0.982439,0.205357,0.694444,0.28,0.0,0.380952,0.92,...,0.432877,0.423077,0.608696,1.000000,0.5,0.621053,1.0,1.496933e+18,0.692003,0.498607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-07 22:45:00,174.8,0.740741,0.83,0.983415,0.080357,0.138889,0.08,0.0,0.000000,0.76,...,0.430137,0.423077,0.956522,1.000000,0.0,0.957895,0.0,1.623106e+18,0.758451,0.288997
2021-06-07 23:00:00,181.6,0.703704,0.87,0.983415,0.169643,0.055556,0.12,0.0,0.000000,0.76,...,0.430137,0.423077,1.000000,0.000000,0.0,0.968421,0.0,1.623107e+18,0.757680,0.300836
2021-06-07 23:15:00,168.8,0.703704,0.87,0.983415,0.169643,0.055556,0.12,0.0,0.000000,0.76,...,0.430137,0.423077,1.000000,0.333333,0.0,0.978947,0.0,1.623108e+18,0.757680,0.278552


In [ ]:
# 設定要選取的欄位，此參數用意在確保所有所需要的欄位與欄位的順序一致 [B]
selected_column = ['Temperature', 'humd', 'pres', 'w_DSD', 'w_DIR', 'h_FX', '24R', 'h_UVI', 'T_Min', 'T_Max', 'day_of_year', 'data_week', 
                   'data_hour', 'data_minute', 'data_weekday', 'minutes_of_the_day', 'apparent_temp']
selected_kmeans_column = ['Temperature', 'humd', 'pres', 'w_DSD', 'w_DIR', 'h_FX', '24R', 'h_UVI', 'T_Min', 'T_Max', 'day_of_year', 'data_week', 
                   'data_hour', 'data_minute', 'data_weekday', 'minutes_of_the_day', 'period_type', 'apparent_temp']
# 設定要選取的欄位，此參數用意在確保所有所需要的欄位與欄位的順序一致 [E]
# 將資料拆分成訓練集語測式集 [B]
all_train_features = demand_dataframe.loc[train_begin_date:train_end_date, selected_column]
all_kmeans_train_features = demand_dataframe.loc[train_begin_date:train_end_date, selected_kmeans_column]
all_train_labels = demand_dataframe.loc[train_begin_date:train_end_date, 'demand_quarter_1']

all_test_features = demand_dataframe.loc[test_begin_date:test_end_date, selected_column]
all_kmeans_test_features = demand_dataframe.loc[test_begin_date:test_end_date, selected_kmeans_column]
all_test_labels = demand_dataframe.loc[test_begin_date:test_end_date, 'demand_quarter_1']

all_predict_features = demand_dataframe.loc[predict_begin_date:predict_end_date, selected_column]
all_kmeans_predict_features = demand_dataframe.loc[predict_begin_date:predict_end_date, selected_kmeans_column]
all_predict_labels = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter_1']
# 將資料拆分成訓練集語測式集 [E]

In [ ]:
# LSTM 模型需放入三維陣列，因此需先將 dataframe 轉為陣列 [B]
all_train_features = np.array(all_train_features)
all_kmeans_train_features = np.array(all_kmeans_train_features)
all_test_features = np.array(all_test_features)
all_kmeans_test_features = np.array(all_kmeans_test_features)
all_predict_features = np.array(all_predict_features)
all_kmeans_predict_features = np.array(all_kmeans_predict_features)
# LSTM 模型需放入三維陣列，因此需先將 dataframe 轉為陣列 [E]

In [ ]:
# 轉為三維陣列 [B]
train_features = np.reshape(all_train_features, (all_train_features.shape[0], all_train_features.shape[1], 1))
test_features = np.reshape(all_test_features, (all_test_features.shape[0], all_test_features.shape[1], 1))
predict_features = np.reshape(all_predict_features, (all_predict_features.shape[0], all_predict_features.shape[1], 1))

train_features2 = np.reshape(all_train_features, (all_train_features.shape[0], 1, all_train_features.shape[1]))
test_features2 = np.reshape(all_test_features, (all_test_features.shape[0], 1, all_test_features.shape[1]))
predict_features2 = np.reshape(all_predict_features, (all_predict_features.shape[0], 1, all_predict_features.shape[1]))

train_kmeans_features2 = np.reshape(all_kmeans_train_features, (all_kmeans_train_features.shape[0], 1, all_kmeans_train_features.shape[1]))
test_kmeans_features2 = np.reshape(all_kmeans_test_features, (all_kmeans_test_features.shape[0], 1, all_kmeans_test_features.shape[1]))
predict_kmeans_features2 = np.reshape(all_kmeans_predict_features, (all_kmeans_predict_features.shape[0], 1, all_kmeans_predict_features.shape[1]))
# 轉為三維陣列 [E]

# 情況 1: 使用 Vanilla LSTM (many to one) 來訓練模型

In [20]:
''' 使用 Vanilla LSTM (many to one) 來訓練模型 [B]'''
# 定義模型架構 [B]
vanilla_model = Sequential()
vanilla_model.add(LSTM(50, input_shape=(train_features2.shape[1],train_features2.shape[2]))) #X_train.shape[1],X_train.shape[2]
vanilla_model.add(Dense(1, activation='relu'))
vanilla_model.compile(loss='mae', optimizer='Adam') 
vanilla_model.summary()
# plot_model(model, to_file='Vanilla_LSTM_graph.png')
# 定義模型架構 [E]
# 開始訓練 [B]
history = vanilla_model.fit(train_features2, all_train_labels, epochs=9000, batch_size=2880, validation_data=(test_features2, all_test_labels), shuffle=False) #(50,72)
# 開始訓練 [E]
# 評估模型 [B]
scores = vanilla_model.evaluate(train_features2, all_train_labels)
# 評估模型 [E]
''' 使用 Vanilla LSTM (many to one) [E]'''

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                13600     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 13,651
Trainable params: 13,651
Non-trainable params: 0
_________________________________________________________________
Epoch 1/9000
23/23 [==============================] - 1s 47ms/step - loss: 0.1465 - val_loss: 0.0950
Epoch 2/9000
23/23 [==============================] - 0s 21ms/step - loss: 0.1084 - val_loss: 0.0845
Epoch 3/9000
23/23 [==============================] - 1s 25ms/step - loss: 0.0991 - val_loss: 0.0804
Epoch 4/9000
23/23 [==============================] - 1s 24ms/step - loss: 0.0956 - val_loss: 0.0770
Epoch 5/9000
23/23 [==============================] - 1s 26ms/step - loss: 0.0930 - val_loss: 0.0749
Epoch 6/9

Epoch 155/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0729 - val_loss: 0.0581
Epoch 156/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0729 - val_loss: 0.0581
Epoch 157/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0728 - val_loss: 0.0580
Epoch 158/9000
23/23 [==============================] - 0s 18ms/step - loss: 0.0727 - val_loss: 0.0580
Epoch 159/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0727 - val_loss: 0.0579
Epoch 160/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0727 - val_loss: 0.0579
Epoch 161/9000
23/23 [==============================] - 0s 18ms/step - loss: 0.0726 - val_loss: 0.0579
Epoch 162/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0725 - val_loss: 0.0578
Epoch 163/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0725 - val_loss: 0.0578
Epoch 164/9000
23/23 [==============================] - 0s 18ms/step - lo

23/23 [==============================] - 0s 19ms/step - loss: 0.0641 - val_loss: 0.0510
Epoch 314/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0641 - val_loss: 0.0509
Epoch 315/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0640 - val_loss: 0.0509
Epoch 316/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0639 - val_loss: 0.0508
Epoch 317/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0639 - val_loss: 0.0507
Epoch 318/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0638 - val_loss: 0.0507
Epoch 319/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0637 - val_loss: 0.0506
Epoch 320/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0636 - val_loss: 0.0505
Epoch 321/9000
23/23 [==============================] - 0s 19ms/step - loss: 0.0635 - val_loss: 0.0505
Epoch 322/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0635 - va

23/23 [==============================] - 0s 17ms/step - loss: 0.0576 - val_loss: 0.0455
Epoch 472/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0576 - val_loss: 0.0455
Epoch 473/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0576 - val_loss: 0.0455
Epoch 474/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0575 - val_loss: 0.0455
Epoch 475/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0575 - val_loss: 0.0455
Epoch 476/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0575 - val_loss: 0.0454
Epoch 477/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0575 - val_loss: 0.0454
Epoch 478/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0574 - val_loss: 0.0454
Epoch 479/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0574 - val_loss: 0.0453
Epoch 480/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0573 - va

23/23 [==============================] - 0s 20ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 630/9000
23/23 [==============================] - 0s 18ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 631/9000
23/23 [==============================] - 1s 22ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 632/9000
23/23 [==============================] - 0s 19ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 633/9000
23/23 [==============================] - 0s 19ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 634/9000
23/23 [==============================] - 0s 20ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 635/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 636/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0549 - val_loss: 0.0436
Epoch 637/9000
23/23 [==============================] - 0s 21ms/step - loss: 0.0548 - val_loss: 0.0436
Epoch 638/9000
23/23 [==============================] - ETA: 0s - loss: 0.057 - 0s 16ms/

...

Epoch 3073/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0462 - val_loss: 0.0392
Epoch 3074/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0461 - val_loss: 0.0392
Epoch 3075/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0463 - val_loss: 0.0392
Epoch 3076/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0461 - val_loss: 0.0392
Epoch 3077/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0461 - val_loss: 0.0393
Epoch 3078/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0463 - val_loss: 0.0392
Epoch 3079/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0461 - val_loss: 0.0392
Epoch 3080/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0462 - val_loss: 0.0393
Epoch 3081/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0462 - val_loss: 0.0392
Epoch 3082/9000
23/23 [==============================] - 0s 17ms

Epoch 3152/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0469 - val_loss: 0.0395
Epoch 3153/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0470 - val_loss: 0.0392
Epoch 3154/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0468 - val_loss: 0.0392
Epoch 3155/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0470 - val_loss: 0.0395
Epoch 3156/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0469 - val_loss: 0.0391
Epoch 3157/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0468 - val_loss: 0.0393
Epoch 3158/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0470 - val_loss: 0.0394
Epoch 3159/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0468 - val_loss: 0.0391
Epoch 3160/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0468 - val_loss: 0.0394
Epoch 3161/9000
23/23 [==============================] - 0s 14ms

Epoch 3231/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0464 - val_loss: 0.0390
Epoch 3232/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0465 - val_loss: 0.0391
Epoch 3233/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0464 - val_loss: 0.0391
Epoch 3234/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0463 - val_loss: 0.0390
Epoch 3235/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0464 - val_loss: 0.0391
Epoch 3236/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0465 - val_loss: 0.0391
Epoch 3237/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0465 - val_loss: 0.0391
Epoch 3238/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0464 - val_loss: 0.0390
Epoch 3239/9000
23/23 [==============================] - 0s 19ms/step - loss: 0.0465 - val_loss: 0.0391
Epoch 3240/9000
23/23 [==============================] - 0s 14ms

Epoch 3310/9000
23/23 [==============================] - 0s 17ms/step - loss: 0.0469 - val_loss: 0.0395
Epoch 3311/9000
23/23 [==============================] - 0s 20ms/step - loss: 0.0469 - val_loss: 0.0393
Epoch 3312/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0467 - val_loss: 0.0391
Epoch 3313/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0467 - val_loss: 0.0392
Epoch 3314/9000
23/23 [==============================] - 0s 14ms/step - loss: 0.0467 - val_loss: 0.0391
Epoch 3315/9000
23/23 [==============================] - 0s 18ms/step - loss: 0.0466 - val_loss: 0.0390
Epoch 3316/9000
23/23 [==============================] - 0s 15ms/step - loss: 0.0466 - val_loss: 0.0391
Epoch 3317/9000
23/23 [==============================] - 0s 16ms/step - loss: 0.0465 - val_loss: 0.0390
Epoch 3318/9000
23/23 [==============================] - 0s 18ms/step - loss: 0.0463 - val_loss: 0.0389
Epoch 3319/9000
23/23 [==============================] - 0s 19ms

KeyboardInterrupt: 

In [ ]:
# 用訓練好的模型來預測 [B]
testPredict1 = vanilla_model.predict(predict_features2)
# 用訓練好的模型來預測 [E]

In [ ]:
# 將預測值 inverse 正規化 [B]
testPredict1 = demand_zscore.inverse_transform(testPredict1)
# 將預測值 inverse 正規化 [E]

In [ ]:
# 抓取實際值 [B]
y_test_true = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter'].array
# 抓取實際值 [E]

In [ ]:
# 將預測值與實際值做圖表 [B]
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([1000,2250])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true, c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict1, c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()
# 將預測值與實際值做圖表 [E]

In [ ]:
# 評估模型好壞 [B]
metrics.mean_absolute_error(y_test_true,testPredict1)
# 評估模型好壞 [E]

In [ ]:
# 儲存模型 [B]
vanilla_model.save(simple_LSTM_modal_save_path)  # creates a HDF5 file 'my_model.h5'
# 儲存模型 [E]

------------------------------------

# 情況 2: 使用 Stacked LSTM (many to one) 來訓練模型

In [ ]:
# Stacked LSTM (many to one) [B]
stacked_model = Sequential()
stacked_model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_features2.shape[1],train_features2.shape[2])))
stacked_model.add(LSTM(50, activation='relu'))
stacked_model.add(Dense(1))
stacked_model.compile(optimizer='adam', loss='mae')
stacked_model.summary()
stacked_model_result = stacked_model.fit(train_features2, all_train_labels, epochs = 1500, batch_size = 2880, validation_data=(test_features2, all_test_labels), shuffle=False) #(50,72)
scores = stacked_model.evaluate(train_features2, all_train_labels)
# Stacked LSTM (many to one) [E]

In [ ]:
testPredict2 = stacked_model.predict(predict_features2)

In [ ]:
testPredict2 = demand_zscore.inverse_transform(testPredict2)

In [ ]:
y_test_true = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter'].array

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([1000,2250])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true, c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict2, c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()

In [ ]:
metrics.mean_absolute_error(y_test_true,testPredict2)

In [ ]:
stacked_model.save(stacked_LSTM_modal_save_path)  # creates a HDF5 file 'my_model.h5'

-----------------------

# 情況 3: 使用 Bidirectional LSTM (many to one) 來訓練模型

In [ ]:
# Bidirectional LSTM (many to one)
bidirectional_model = Sequential()
bidirectional_model.add(Bidirectional(LSTM(50, activation='linear', return_sequences=True), input_shape=(train_features2.shape[1],train_features2.shape[2])))
bidirectional_model.add(Bidirectional(LSTM(50)))
bidirectional_model.add(Dense(1))
bidirectional_model.compile(optimizer='Adam', loss='mse')
bidirectional_model.summary()
bidirectional_model_result = bidirectional_model.fit(train_features2, all_train_labels, epochs=600, batch_size = 2880, validation_data=(test_features2, all_test_labels), shuffle=False) #(50,72)
scores = bidirectional_model.evaluate(train_features2, all_train_labels)

In [ ]:
testPredict3 = bidirectional_model.predict(predict_features2)
testPredict3 = demand_zscore.inverse_transform(testPredict3)
y_test_true = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter'].array

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([1000,2250])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true, c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict3, c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()

In [ ]:
metrics.mean_absolute_error(y_test_true,testPredict3)

In [ ]:
bidirectional_model.save(bidirectional_LSTM_modal_save_path)  # creates a HDF5 file 'my_model.h5'

------------

# 情況 4: 使用 Vanilla LSTM (many to one) 並加入 k-means 的分群結果來訓練模型

In [ ]:
# Vanilla kmeans LSTM (many to one)
vanilla_kmeans_model = Sequential()
vanilla_kmeans_model.add(LSTM(50, input_shape=(train_kmeans_features2.shape[1],train_kmeans_features2.shape[2]))) #X_train.shape[1],X_train.shape[2]
vanilla_kmeans_model.add(Dense(1, activation='relu'))
vanilla_kmeans_model.compile(loss='mae', optimizer='Adam') 
vanilla_kmeans_model.summary()
# plot_model(model, to_file='Vanilla_LSTM_graph.png')
history = vanilla_kmeans_model.fit(train_kmeans_features2, all_train_labels, epochs=9000, batch_size=2880, validation_data=(test_kmeans_features2, all_test_labels), shuffle=False) #(50,72)
scores = vanilla_kmeans_model.evaluate(train_kmeans_features2, all_train_labels)


In [ ]:
testPredict_kmeans_1 = vanilla_kmeans_model.predict(predict_kmeans_features2)

In [ ]:
testPredict_kmeans_1 = demand_zscore.inverse_transform(testPredict_kmeans_1)

In [ ]:
y_test_true = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter'].array

In [ ]:

from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([1000,2250])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true, c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict_kmeans_1, c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()


In [ ]:
#評估模型好壞
#載入迴歸常見的評估指標
metrics.mean_absolute_error(y_test_true,testPredict_kmeans_1)

In [ ]:
vanilla_kmeans_model.save(simple_LSTM_kmeans_modal_save_path)  # creates a HDF5 file 'my_model.h5'

--------------

In [ ]:
# Stacked LSTM (many to one) [B]
stacked_kmeans_model = Sequential()
stacked_kmeans_model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_kmeans_features2.shape[1],train_kmeans_features2.shape[2])))
stacked_kmeans_model.add(LSTM(50, activation='relu'))
stacked_kmeans_model.add(Dense(1))
stacked_kmeans_model.compile(optimizer='adam', loss='mae')
stacked_kmeans_model.summary()
stacked_kmeans_model_result = stacked_kmeans_model.fit(train_kmeans_features2, all_train_labels, epochs = 1500, batch_size = 2880, validation_data=(test_kmeans_features2, all_test_labels), shuffle=False) #(50,72)
scores = stacked_kmeans_model.evaluate(train_kmeans_features2, all_train_labels)
# Stacked LSTM (many to one) [E]


In [ ]:
testPredict_kmeans_2 = stacked_kmeans_model.predict(predict_kmeans_features2)

In [ ]:
testPredict_kmeans_2 = demand_zscore.inverse_transform(testPredict_kmeans_2)

In [ ]:
y_test_true = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter'].array

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([1000,2250])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true, c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict_kmeans_2, c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()

In [ ]:
#評估模型好壞
#載入迴歸常見的評估指標
metrics.mean_absolute_error(y_test_true,testPredict_kmeans_2)

In [ ]:
stacked_kmeans_model.save(stacked_LSTM_kmeans_modal_save_path)  # creates a HDF5 file 'my_model.h5'

---------------

In [ ]:
# Stacked LSTM (many to one) [B]
bidirectional_kmeans_model = Sequential()
bidirectional_kmeans_model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(train_kmeans_features2.shape[1],train_kmeans_features2.shape[2])))
bidirectional_kmeans_model.add(LSTM(50, activation='relu'))
bidirectional_kmeans_model.add(Dense(1))
bidirectional_kmeans_model.compile(optimizer='adam', loss='mae')
bidirectional_kmeans_model.summary()
bidirectional_kmeans_model_result = bidirectional_kmeans_model.fit(train_kmeans_features2, all_train_labels, epochs = 1500, batch_size = 2880, validation_data=(test_kmeans_features2, all_test_labels), shuffle=False) #(50,72)
scores = bidirectional_kmeans_model.evaluate(train_kmeans_features2, all_train_labels)
# Stacked LSTM (many to one) [E]

In [ ]:
testPredict_kmeans_3 = bidirectional_kmeans_model.predict(predict_kmeans_features2)

In [ ]:
testPredict_kmeans_3 = demand_zscore.inverse_transform(testPredict_kmeans_3)

In [ ]:
y_test_true = demand_dataframe.loc[predict_begin_date:predict_end_date, 'demand_quarter'].array

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(15,6))
plt.xlim([1000,2250])
plt.ylabel("demand")
plt.xlabel("minutes")
plt.plot(y_test_true, c = "g", alpha = 0.90, linewidth = 2.5)
plt.plot(testPredict_kmeans_3, c = "darkblue", alpha = 0.75)
plt.title("Testing(Validation) data")
plt.show()

In [ ]:
#評估模型好壞
#載入迴歸常見的評估指標
metrics.mean_absolute_error(y_test_true,testPredict_kmeans_3)


In [ ]:
bidirectional_kmeans_model.save(bidirectional_LSTM_kmeans_modal_save_path)  # creates a HDF5 file 'my_model.h5'

------------

In [ ]:
# 儲存檔案最後更新檔案日期 [B]
vanilla_modal_pid_write = open(modal_pid_path, 'w', encoding='utf-8')
vanilla_modal_pid_write.write(today)
stacked_modal_pid_write = open(stacked_modal_pid_path, 'w', encoding='utf-8')
stacked_modal_pid_write.write(today)
bidirectional_model_pid_write = open(bidirectional_modal_pid_path, 'w', encoding='utf-8')
bidirectional_model_pid_write.write(today)
simple_kmeans_modal_pid_write = open(simple_kmeans_modal_pid_path, 'w', encoding='utf-8')
simple_kmeans_modal_pid_write.write(today)
stacked_kmeans_modal_pid_write = open(stacked_kmeans_modal_pid_path, 'w', encoding='utf-8')
stacked_kmeans_modal_pid_write.write(today)
bidirectional_kmeans_model_pid_write = open(bidirectional_kmeans_modal_pid_path, 'w', encoding='utf-8')
bidirectional_kmeans_model_pid_write.write(today)
print(today)
# 儲存檔案最後更新檔案日期 [E]

In [ ]:
del model  # deletes the existing model